In [6]:
# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

import grpc
from concurrent import futures
# import the generated classes
import handle_new_frame_pb2
import handle_new_frame_pb2_grpc
import base64

# For downloading the image.
import matplotlib.pyplot as plt

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time
import threading
import sys
import psutil
import logging
import cv2

In [4]:
def save_image(image, image_name):
    Image.fromarray(image).save(image_name)

def resize_image(raw_frame, new_width, new_height):

    pil_image = Image.fromarray(np.uint8(raw_frame))
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")

    img = tf.convert_to_tensor(pil_image_rgb, dtype=tf.uint8)
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    return converted_img


In [15]:
video_path = '../videos/Rec_20200125_170152_211_S.mp4'

# Define a video capture object
cap = cv2.VideoCapture(video_path)


ret, frame = cap.read()

img = resize_image(frame, 1280, 856)


print(type(img))
print(type(frame))
print(frame.shape)

boh = img.numpy()
boh.reshape(856, 1280, 3)
print(type(boh))
print(boh.shape)
#save_image(img.numpy(), "tensore_salvato.jpg")

cap.release()

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
(352, 640, 3)
<class 'numpy.ndarray'>
(1, 856, 1280, 3)


In [4]:
i = 5

if i not in range(1, 5):
    print("Non è nel range")

else:
    print("E' nel range")


for j in range(1, i + 1):
    print(j)

Non è nel range
1
2
3
4
5


In [10]:
import numpy as np
import json

def decode_result(result):
    # to make it serializable
    for i, v in enumerate(result['detection_class_entities']):
        result['detection_class_entities'][i] = v.decode('utf-8')

    for i, v in enumerate(result['detection_class_names']):
        result['detection_class_names'][i] = v.decode('utf-8')
    
    return result


result = {'detection_class_labels': np.array([571, 571, 571, 409,  69, 241, 409, 409, 571, 391, 409, 571, 391,
       571, 391, 391, 462, 462, 391,  69,  69, 571, 552, 571, 241, 433,
       391, 391, 391, 552, 391, 308, 391, 552, 391, 571, 391, 391, 391,
       391, 391, 391, 391, 391, 552, 391, 391, 241, 103, 308, 391, 308,
       433, 571, 433, 103, 571, 571, 131, 391, 571, 391, 571, 103, 103,
       391, 103, 103, 391, 103, 404, 433, 298, 103, 571, 103, 404, 462,
       434, 404, 391, 103, 571, 409, 409, 445, 409, 391,  69,  69, 552,
       391, 571, 409, 391, 404, 404, 103, 103, 459]), 'detection_class_names': np.array([b'/m/0k4j', b'/m/0k4j', b'/m/0k4j', b'/m/083wq', b'/m/01g317',
       b'/m/03jm5', b'/m/083wq', b'/m/083wq', b'/m/0k4j', b'/m/07j7r',
       b'/m/083wq', b'/m/0k4j', b'/m/07j7r', b'/m/0k4j', b'/m/07j7r',
       b'/m/07j7r', b'/m/0cgh4', b'/m/0cgh4', b'/m/07j7r', b'/m/01g317',
       b'/m/01g317', b'/m/0k4j', b'/m/0h9mv', b'/m/0k4j', b'/m/03jm5',
       b'/m/09j2d', b'/m/07j7r', b'/m/07j7r', b'/m/07j7r', b'/m/0h9mv',
       b'/m/07j7r', b'/m/04yx4', b'/m/07j7r', b'/m/0h9mv', b'/m/07j7r',
       b'/m/0k4j', b'/m/07j7r', b'/m/07j7r', b'/m/07j7r', b'/m/07j7r',
       b'/m/07j7r', b'/m/07j7r', b'/m/07j7r', b'/m/07j7r', b'/m/0h9mv',
       b'/m/07j7r', b'/m/07j7r', b'/m/03jm5', b'/m/01prls', b'/m/04yx4',
       b'/m/07j7r', b'/m/04yx4', b'/m/09j2d', b'/m/0k4j', b'/m/09j2d',
       b'/m/01prls', b'/m/0k4j', b'/m/0k4j', b'/m/0220r2', b'/m/07j7r',
       b'/m/0k4j', b'/m/07j7r', b'/m/0k4j', b'/m/01prls', b'/m/01prls',
       b'/m/07j7r', b'/m/01prls', b'/m/01prls', b'/m/07j7r', b'/m/01prls',
       b'/m/07yv9', b'/m/09j2d', b'/m/04rky', b'/m/01prls', b'/m/0k4j',
       b'/m/01prls', b'/m/07yv9', b'/m/0cgh4', b'/m/09j5n', b'/m/07yv9',
       b'/m/07j7r', b'/m/01prls', b'/m/0k4j', b'/m/083wq', b'/m/083wq',
       b'/m/0b_rs', b'/m/083wq', b'/m/07j7r', b'/m/01g317', b'/m/01g317',
       b'/m/0h9mv', b'/m/07j7r', b'/m/0k4j', b'/m/083wq', b'/m/07j7r',
       b'/m/07yv9', b'/m/07yv9', b'/m/01prls', b'/m/01prls', b'/m/0cdl1'],
      dtype=object), 'detection_class_entities': np.array([b'Car', b'Car', b'Car', b'Wheel', b'Person', b'House', b'Wheel',
       b'Wheel', b'Car', b'Tree', b'Wheel', b'Car', b'Tree', b'Car',
       b'Tree', b'Tree', b'Building', b'Building', b'Tree', b'Person',
       b'Person', b'Car', b'Tire', b'Car', b'House', b'Clothing', b'Tree',
       b'Tree', b'Tree', b'Tire', b'Tree', b'Man', b'Tree', b'Tire',
       b'Tree', b'Car', b'Tree', b'Tree', b'Tree', b'Tree', b'Tree',
       b'Tree', b'Tree', b'Tree', b'Tire', b'Tree', b'Tree', b'House',
       b'Land vehicle', b'Man', b'Tree', b'Man', b'Clothing', b'Car',
       b'Clothing', b'Land vehicle', b'Car', b'Car', b'Fountain', b'Tree',
       b'Car', b'Tree', b'Car', b'Land vehicle', b'Land vehicle', b'Tree',
       b'Land vehicle', b'Land vehicle', b'Tree', b'Land vehicle',
       b'Vehicle', b'Clothing', b'Mammal', b'Land vehicle', b'Car',
       b'Land vehicle', b'Vehicle', b'Building', b'Footwear', b'Vehicle',
       b'Tree', b'Land vehicle', b'Car', b'Wheel', b'Wheel',
       b'Swimming pool', b'Wheel', b'Tree', b'Person', b'Person', b'Tire',
       b'Tree', b'Car', b'Wheel', b'Tree', b'Vehicle', b'Vehicle',
       b'Land vehicle', b'Land vehicle', b'Palm tree'], dtype=object), 'detection_boxes': np.array([[3.33016306e-01, 8.12450722e-02, 5.61993241e-01, 3.67244244e-01],
       [3.51117432e-01, 8.02130718e-03, 4.02606130e-01, 5.67796044e-02],
       [3.30808103e-01, 3.36201906e-01, 4.55026507e-01, 5.12085974e-01],
       [4.68793958e-01, 2.25589946e-01, 5.56948781e-01, 2.78991997e-01],
       [3.00341398e-01, 1.10129945e-01, 6.10144854e-01, 1.72536701e-01],
       [1.55249611e-01, 5.91421843e-01, 3.58537287e-01, 8.49058092e-01],
       [3.97280693e-01, 3.51260334e-01, 4.44413304e-01, 3.79737079e-01],
       [4.54426855e-01, 8.28633383e-02, 5.20155489e-01, 1.14537850e-01],
       [3.45072508e-01, 7.18103647e-02, 5.17319441e-01, 2.04952300e-01],
       [1.11116692e-01, 9.55141068e-01, 2.88632274e-01, 9.97883141e-01],
       [4.10049856e-01, 4.65517402e-01, 4.57870483e-01, 4.95525062e-01],
       [3.34472418e-01, 4.57381070e-01, 4.23697382e-01, 5.58430314e-01],
       [3.28579322e-02, 7.07172096e-01, 1.69838205e-01, 7.68448174e-01],
       [3.43881756e-01, 2.98590928e-01, 3.81349683e-01, 3.54479134e-01],
       [7.38971829e-02, 7.83003569e-01, 1.90836862e-01, 8.91556740e-01],
       [1.11119196e-01, 6.62188411e-01, 1.77643955e-01, 6.98000848e-01],
       [1.51353121e-01, 5.93997002e-01, 3.55970800e-01, 8.91474187e-01],
       [1.88850254e-01, 8.31463456e-01, 3.45866710e-01, 9.93026018e-01],
       [1.05226077e-01, 8.71391892e-01, 3.89068663e-01, 9.68803167e-01],
       [2.84837335e-01, 1.14177883e-01, 4.82236028e-01, 1.64825499e-01],
       [2.84982145e-01, 1.15249433e-01, 4.15806025e-01, 1.60292313e-01],
       [3.32833141e-01, 4.62285906e-01, 3.95461202e-01, 5.51566124e-01],
       [3.96532744e-01, 3.51072967e-01, 4.45233822e-01, 3.79738867e-01],
       [3.37985396e-01, 2.95080423e-01, 4.02690381e-01, 3.64552796e-01],
       [1.83659464e-01, 8.35542142e-01, 3.48406941e-01, 9.92383838e-01],
       [3.09107572e-01, 1.11887857e-01, 5.90824068e-01, 1.66021809e-01],
       [5.60070053e-02, 7.57057071e-01, 1.62860110e-01, 8.00783575e-01],
       [6.97315251e-03, 3.13471526e-01, 3.29180866e-01, 5.21225631e-01],
       [4.48515713e-02, 7.29130924e-01, 1.64708093e-01, 7.99281478e-01],
       [4.55555081e-01, 8.32847208e-02, 5.19587755e-01, 1.12744167e-01],
       [1.26677096e-01, 4.83701348e-01, 2.67058581e-01, 5.68146467e-01],
       [2.90241897e-01, 1.11972980e-01, 5.42682290e-01, 1.62387699e-01],
       [8.45484063e-02, 5.89377642e-01, 1.79570884e-01, 6.63468659e-01],
       [4.11294043e-01, 4.65089411e-01, 4.59542364e-01, 4.95865136e-01],
       [3.69724669e-02, 6.96348965e-01, 1.83888569e-01, 8.33578706e-01],
       [3.62615764e-01, 8.77220882e-05, 4.30552781e-01, 3.18480618e-02],
       [8.81846324e-02, 5.44972062e-01, 2.71305919e-01, 6.73541486e-01],
       [3.67952622e-02, 1.73296558e-03, 3.36669981e-01, 7.68685043e-02],
       [1.07899606e-02, 5.36285602e-02, 2.86896467e-01, 1.47906408e-01],
       [8.18608999e-02, 8.90074551e-01, 1.95791006e-01, 9.32969391e-01],
       [1.20000154e-01, 9.75765288e-01, 2.87310362e-01, 9.95439410e-01],
       [7.42039084e-02, 8.35968196e-01, 1.94197983e-01, 8.95866215e-01],
       [9.13471207e-02, 8.90569985e-01, 3.21086317e-01, 9.33375835e-01],
       [7.80564770e-02, 7.80909777e-01, 1.84594318e-01, 8.49203885e-01],
       [4.67491657e-01, 2.26888970e-01, 5.55105925e-01, 2.76832283e-01],
       [1.74618647e-01, 8.43087077e-01, 3.57859105e-01, 9.85913694e-01],
       [6.62497198e-03, 1.53551638e-01, 2.84905553e-01, 2.10020080e-01],
       [1.51084557e-01, 7.90602326e-01, 3.68625760e-01, 8.51125658e-01],
       [3.35100114e-01, 4.57151979e-01, 4.31988627e-01, 5.54652810e-01],
       [2.85878807e-01, 1.15398876e-01, 4.12913978e-01, 1.59629524e-01],
       [8.06627870e-02, 8.77522528e-01, 1.96832791e-01, 9.21281695e-01],
       [2.91985035e-01, 1.07750922e-01, 6.09957337e-01, 1.74041122e-01],
       [3.07939351e-01, 1.15096442e-01, 4.69131649e-01, 1.64531529e-01],
       [3.27183723e-01, 4.60538357e-01, 3.75047147e-01, 5.46461165e-01],
       [3.06181401e-01, 1.16013072e-01, 3.98647457e-01, 1.61147654e-01],
       [3.37137908e-01, 6.55420944e-02, 5.58919013e-01, 3.70372951e-01],
       [3.46596271e-01, 4.68412250e-01, 4.46575433e-01, 5.40736496e-01],
       [3.59932691e-01, 7.06004677e-03, 4.08842504e-01, 2.82970890e-02],
       [2.67466247e-01, 0.00000000e+00, 9.97564912e-01, 7.41545975e-01],
       [4.64974716e-03, 2.65033960e-01, 2.60510266e-01, 4.12835389e-01],
       [3.32446307e-01, 4.49554354e-01, 4.39990193e-01, 6.07441187e-01],
       [7.96113610e-02, 7.82318711e-01, 3.12162876e-01, 8.98757458e-01],
       [3.39081079e-01, 3.08058858e-01, 3.66528839e-01, 3.61158699e-01],
       [3.30895334e-01, 3.34275454e-01, 4.56404835e-01, 5.09851992e-01],
       [3.33317935e-01, 4.49202895e-01, 4.39201564e-01, 6.08039618e-01],
       [8.57035294e-02, 8.57569218e-01, 1.97217822e-01, 9.13522482e-01],
       [3.04635733e-01, 9.10846770e-01, 3.69033724e-01, 9.68310595e-01],
       [3.34191501e-01, 4.62560385e-01, 3.93411607e-01, 5.52341104e-01],
       [6.71939878e-03, 2.34824300e-01, 2.74260908e-01, 3.46426040e-01],
       [3.52351606e-01, 8.47698655e-03, 4.04710233e-01, 5.60368486e-02],
       [3.36812437e-01, 4.59093958e-01, 4.31352437e-01, 5.52703440e-01],
       [4.66944128e-01, 1.14243783e-01, 6.04206324e-01, 1.62175596e-01],
       [2.92662263e-01, 1.11349680e-01, 6.11944914e-01, 1.69341907e-01],
       [3.41907293e-01, 7.27986470e-02, 5.32986403e-01, 2.04858005e-01],
       [3.55307370e-01, 1.12930126e-02, 3.94940972e-01, 2.51672082e-02],
       [3.27784419e-01, 6.11816704e-01, 3.60926330e-01, 6.56813443e-01],
       [3.26717615e-01, 6.10128403e-01, 3.61229599e-01, 6.56336188e-01],
       [1.56588882e-01, 7.91222751e-01, 3.61874789e-01, 8.50533962e-01],
       [5.62629938e-01, 1.41297475e-01, 6.09979510e-01, 1.71477556e-01],
       [3.35658580e-01, 4.49626833e-01, 4.35073674e-01, 6.01091504e-01],
       [1.07633978e-01, 7.81547010e-01, 3.38720262e-01, 8.65527034e-01],
       [3.59487504e-01, 6.49106689e-04, 4.34508741e-01, 3.17385979e-02],
       [3.53096992e-01, 7.33540207e-02, 5.12237310e-01, 1.33873969e-01],
       [3.88024747e-01, 1.11304009e-02, 4.44498599e-01, 2.22546645e-02],
       [4.88563180e-01, 3.27222824e-01, 5.19221485e-01, 3.55854034e-01],
       [4.35015887e-01, 0.00000000e+00, 9.63834345e-01, 7.64126003e-01],
       [3.93982232e-01, 4.97817636e-01, 4.44300354e-01, 5.23295641e-01],
       [1.04126550e-01, 6.29336894e-01, 1.78488046e-01, 6.78806543e-01],
       [4.54143137e-01, 1.11004211e-01, 6.14520967e-01, 1.62836120e-01],
       [3.08481753e-01, 9.05840769e-02, 5.66559434e-01, 1.84248745e-01],
       [3.94015789e-01, 4.96840298e-01, 4.41968650e-01, 5.23815751e-01],
       [4.67918394e-03, 1.66730478e-01, 2.35933915e-01, 2.05950841e-01],
       [3.61929208e-01, 1.03362072e-02, 3.96862626e-01, 3.44479643e-02],
       [3.95071030e-01, 4.70157295e-01, 4.60624963e-01, 5.07661879e-01],
       [7.99827725e-02, 6.20100677e-01, 3.59852195e-01, 9.41377759e-01],
       [3.50992143e-01, 5.46274427e-03, 3.99530232e-01, 5.67256920e-02],
       [3.31578881e-01, 3.37078333e-01, 4.55070466e-01, 5.10250628e-01],
       [3.45514446e-01, 4.69274014e-01, 4.45851862e-01, 5.40173709e-01],
       [3.39675367e-01, 4.96007174e-01, 4.27904457e-01, 6.03184342e-01],
       [8.91176984e-02, 8.89985621e-01, 3.15717220e-01, 9.34276462e-01]],
      dtype=np.float32), 'detection_scores': np.array([0.92942375, 0.8048842 , 0.76273566, 0.6015691 , 0.58990437,
       0.48618945, 0.46562713, 0.3863645 , 0.35009167, 0.3396728 ,
       0.30552986, 0.29895052, 0.25044316, 0.23806156, 0.22173265,
       0.16935007, 0.16663529, 0.161778  , 0.15984797, 0.15347151,
       0.14964226, 0.14398992, 0.14338914, 0.13735878, 0.13532214,
       0.13258845, 0.13246326, 0.12818015, 0.11775664, 0.11666382,
       0.11174808, 0.11015655, 0.10702661, 0.10453755, 0.10160662,
       0.09857692, 0.09733497, 0.09617122, 0.09000741, 0.08552591,
       0.08509153, 0.08444954, 0.08280522, 0.07924517, 0.07811595,
       0.07211202, 0.0699935 , 0.06837131, 0.05982611, 0.05937991,
       0.05616608, 0.05173432, 0.0510216 , 0.05038426, 0.04761264,
       0.04370527, 0.04174443, 0.04157146, 0.03932124, 0.0380755 ,
       0.03772416, 0.03760241, 0.03183654, 0.03146082, 0.03011403,
       0.02845273, 0.02732419, 0.02683973, 0.02544436, 0.02220659,
       0.02180922, 0.02145174, 0.01954398, 0.01926064, 0.01825623,
       0.01755616, 0.01665006, 0.0163564 , 0.01627979, 0.01575189,
       0.0153703 , 0.01501211, 0.01460527, 0.01459081, 0.01427864,
       0.01366997, 0.01315714, 0.01300272, 0.01289433, 0.01229255,
       0.0120772 , 0.01140168, 0.01117199, 0.01115515, 0.01110424,
       0.01107012, 0.0110675 , 0.01104716, 0.01102456, 0.01093495],
      dtype=np.float32)}

detection_class_labels = result['detection_class_labels']
detection_class_names = result['detection_class_names']
detection_class_entities = result['detection_class_entities']
detection_boxes = result['detection_boxes']
detection_scores = result['detection_scores']


dict_to_save = {'detection_class_entities': result['detection_class_entities'], 'detection_scores': result['detection_scores']}

print(f"dizionario: {dict_to_save}")


dizionario: {'detection_class_entities': array([b'Car', b'Car', b'Car', b'Wheel', b'Person', b'House', b'Wheel',
       b'Wheel', b'Car', b'Tree', b'Wheel', b'Car', b'Tree', b'Car',
       b'Tree', b'Tree', b'Building', b'Building', b'Tree', b'Person',
       b'Person', b'Car', b'Tire', b'Car', b'House', b'Clothing', b'Tree',
       b'Tree', b'Tree', b'Tire', b'Tree', b'Man', b'Tree', b'Tire',
       b'Tree', b'Car', b'Tree', b'Tree', b'Tree', b'Tree', b'Tree',
       b'Tree', b'Tree', b'Tree', b'Tire', b'Tree', b'Tree', b'House',
       b'Land vehicle', b'Man', b'Tree', b'Man', b'Clothing', b'Car',
       b'Clothing', b'Land vehicle', b'Car', b'Car', b'Fountain', b'Tree',
       b'Car', b'Tree', b'Car', b'Land vehicle', b'Land vehicle', b'Tree',
       b'Land vehicle', b'Land vehicle', b'Tree', b'Land vehicle',
       b'Vehicle', b'Clothing', b'Mammal', b'Land vehicle', b'Car',
       b'Land vehicle', b'Vehicle', b'Building', b'Footwear', b'Vehicle',
       b'Tree', b'Land vehicle', b

In [20]:

import time
value = time.time_ns()

print(type(value))
print(f'{value:,}')
time.sleep(1)
value2 = time.time_ns()

print(f'{value2-value:,}')



<class 'int'>
1,637,673,203,943,706,300
1,000,654,300


In [54]:

import time

def current_time_int():
    return int(round(time.time() * 1000_000_000))


n = time.time_ns()
t = current_time_int()

print(t)
print(n)
print(n - t)


1637691357933887744
1637691357933887700
-44


In [58]:

# CREATION_TS: 1637691737260139776 SERVICE_TS: 1637691737424230400 COMPLETION_TS: 1637691738735260928

c = 1637691737260139776 / 1000_000_000
s = 1637691737424230400 / 1000_000_000
f = 1637691738735260928 / 1000_000_000


print(f"{c:,}")
print(f"{s:,}")
print(f"{f:,}")

print(f"{s-c:,}")
print(f"{f-s:,}")
print(f"{f-c:,}")


1,637,691,737.2601397
1,637,691,737.4242303
1,637,691,738.735261
0.16409063339233398
1.311030626296997
1.475121259689331


In [27]:
response = {
    "data": [
        {
            "created_at": "2021-11-23T17:00:20.000Z",
            "id": "1463190693116059649",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/23/2021, 12:00 is: 41.77. It feels like 34.74. Raining Probability 0.0. Rain Intensity : 0.0 The day is Clear #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/ZM1RVSgPlc"
        },
        {
            "created_at": "2021-11-22T23:00:20.000Z",
            "id": "1462918899746357252",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/22/2021, 18:00 is: 41.75. It feels like 35.09. Raining Probability 0.0. Rain Intensity : 0.0 The day is Mostly Cloudy #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/28AUv7Vmap"
        },
        {
            "created_at": "2021-11-22T17:00:19.000Z",
            "id": "1462828302121504768",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/22/2021, 12:00 is: 51.01. It feels like 51.01. Raining Probability 0.14. Rain Intensity : 0.0019 The day is Overcast #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https://t.co/dX1jmhWEpT"
        },
        {
            "created_at": "2021-11-22T11:00:19.000Z",
            "id": "1462737705243226117",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/22/2021, 06:00 is: 46.07. It feels like 42.53. Raining Probability 1.0. Rain Intensity : 0.0072 The day is Drizzle #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/LOP7H3KkvL"
        },
        {
            "created_at": "2021-11-21T17:00:20.000Z",
            "id": "1462465917665923078",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/21/2021, 12:00 is: 51.43. It feels like 51.43. Raining Probability 0.0. Rain Intensity : 0.0 The day is Partly Cloudy #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/hbfZSI2Ojv"
        },
        {
            "created_at": "2021-11-21T11:00:20.000Z",
            "id": "1462375320707911683",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/21/2021, 06:00 is: 33.1. It feels like 30.24. Raining Probability 0.0. Rain Intensity : 0.0 The day is Mostly Cloudy #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https://t.co/IIJw99PZae"
        },
        {
            "created_at": "2021-11-20T23:00:20.000Z",
            "id": "1462194123755171840",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/20/2021, 18:00 is: 42.15. It feels like 38.18. Raining Probability 0.0. Rain Intensity : 0.0 The day is Overcast #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/0u0OMOmfM8"
        },
        {
            "created_at": "2021-11-19T17:00:20.000Z",
            "id": "1461741139468374019",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/19/2021, 12:00 is: 45.18. It feels like 38.41. Raining Probability 0.0. Rain Intensity : 0.0 The day is Clear #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/gbY7YoATyg"
        },
        {
            "created_at": "2021-11-19T11:00:19.000Z",
            "id": "1461650540459438082",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/19/2021, 06:00 is: 37.83. It feels like 30.58. Raining Probability 0.0. Rain Intensity : 0.0 The day is Clear #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/7bInOtBkND"
        },
        {
            "created_at": "2021-11-18T17:00:19.000Z",
            "id": "1461378750751383558",
            "text": "Current Temp at Hamilton Str. Prkng Lot D on 11/18/2021, 12:00 is: 67.91. It feels like 67.91. Raining Probability 0.0. Rain Intensity : 0.0 The day is Clear #EllicottCity,#Flood,#Flashflood,#Floodbot,#AIConverse #UMBC Prototype/NSF GR#1640625 https: //t.co/KfUw3Eyxg4"
        }
    ],
    "meta": {
        "newest_id": "1463190693116059649",
        "next_token": "7140dibdnow9c7btw3z3an84ri0vfi31iclpdpv3498vs",
        "oldest_id": "1461378750751383558",
        "result_count": 10
    }
}

rain_intesity = "Rain Intensity : "
the_day = "The day is"

for tweet in response['data']:
    text = tweet['text']
    #text = "Double Flood Monitoring #flooding #ellicottcity"
    rain_index = text.find(rain_intesity)
    end_index = text.find(the_day)
    if rain_index == -1 or end_index == -1:
        print("Text not usable")
    else:
        f = float(text[rain_index + len(rain_intesity):end_index])
        print(f)
        v = f > 0.5
        print(v)






0.0
False
0.0
False
0.0019
False
0.0072
False
0.0
False
0.0
False
0.0
False
0.0
False
0.0
False
0.0
False
